<a href="https://colab.research.google.com/github/safikhanSoofiyani/CS6910-Assignment-3/blob/main/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Library imports


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy as sc
import tensorflow as tf
import keras as ks
import random
import csv

from keras.layers import Dense
from keras.layers import Input
from keras.layers import InputLayer
from keras.layers import Flatten
from keras.layers import Activation 
from keras.layers import LSTM 
from keras.layers import SimpleRNN
from keras.layers import GRU 
from keras.layers import TimeDistributed
from keras.layers import Embedding
from keras.models import Sequential, Model, load_model

import tqdm

In [ ]:
random.seed(hash("seriously you compete with me") % 2**32 - 1)
np.random.seed(hash("i am mohammed safi") % 2**32 - 1)
tf.random.set_seed(hash("ur rahman khan") % 2**32 - 1)

In [ ]:
from tensorflow.python.keras import backend as K
K._get_available_gpus()

['/device:GPU:0']

In [ ]:
!nvidia-smi

Fri Apr 29 11:23:28 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    41W / 300W |    471MiB / 16160MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install wandb
import wandb

from wandb.keras import WandbCallback

     |████████████████████████████████| 1.8 MB 14.3 MB/s 
     |████████████████████████████████| 144 kB 70.8 MB/s 
     |████████████████████████████████| 181 kB 70.5 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=62db3f63ea66c023764ec82e8d00bf052a3b7f701d9973b094f9ed6b48e624f2
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data Preperation 
#### Train and Validation dataset

In [ ]:
def load_data():
    path = r"/content/drive/MyDrive/dakshina_dataset_v1.0/dakshina_dataset_v1.0/te/lexicons/"

    train_file_path = path + "te.translit.sampled.train.tsv"
    val_file_path = path + "te.translit.sampled.dev.tsv"

    train_tsv_file = open(train_file_path, "r", encoding="utf-8")
    train_dataset = csv.reader(train_tsv_file, delimiter="\t")

    val_tsv_file = open(val_file_path, "r", encoding="utf-8")
    val_dataset = csv.reader(val_tsv_file, delimiter="\t")

    return train_dataset, val_dataset


In [ ]:
def prepare_data():

    train_dataset, val_dataset = load_data()
    
    input = []
    target = []
    #print(english)
    for i in train_dataset:
        #print(i) 
        target.append(i[0])
        #print(i[1])
        input.append(i[1])
    #print(english)
    target = np.array(target)
    input = np.array(input)

    # Validation data
    val_input = []
    val_target = []

    for i in val_dataset:
        val_target.append(i[0])
        val_input.append(i[1])

    val_target = np.array(val_target)
    val_input = np.array(val_input)

    for i in range(len(target)):
        target[i] = "\t" + target[i] + "\n"
    
    for i in range(len(val_target)):
        val_target[i] = "\t" + val_target[i] + "\n"

    return input, target, val_input, val_target



In [ ]:
def getTokens(input, target, val_input, val_target):
    # Getting input and target language characters

    # Training set
    input_tokens = set()
    target_tokens = set()

    for word in input:
        for char in word:
            if char not in input_tokens:
                input_tokens.add(char)

    for word in target:
        for char in word:
            if char not in target_tokens:
                target_tokens.add(char)

    # Validation set
    val_input_tokens = set()
    val_target_tokens = set()

    for word in val_input:
        for char in word:
            if char not in val_input_tokens:
                val_input_tokens.add(char)

    for word in val_target:
        for char in word:
            if char not in val_target_tokens:
                val_target_tokens.add(char)

    input_tokens = sorted(list(input_tokens))
    target_tokens = sorted(list(target_tokens))
    
    return input_tokens, target_tokens, val_input_tokens, val_target_tokens


In [ ]:
input, target, val_input, val_target = prepare_data()
input_tokens, target_tokens, val_input_tokens, val_target_tokens = getTokens(input, target, val_input, val_target)


In [ ]:
num_encoder_tokens = len(input_tokens)
num_decoder_tokens = len(target_tokens)

max_encoder_seq_length = max([len(txt) for txt in input])
max_decoder_seq_length = max([len(txt) for txt in target])

input_token_index = dict([(char, i) for i, char in enumerate(input_tokens)])
target_token_index = dict([(char, i) for i, char in enumerate(target_tokens)])
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

print("Number of data points:", len(input))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

Number of data points: 58550
Number of unique input tokens: 26
Number of unique output tokens: 65
Max sequence length for inputs: 25
Max sequence length for outputs: 20


In [ ]:
# printing the target train data for encoder
target

array(['\tఅంకిత\n', '\tఅంకిత\n', '\tఅంకిత\n', ..., '\tహ్యూమన్\n',
       '\tహ్యూమన్\n', '\tహ్యూమన్\n'], dtype='<U20')

In [ ]:
# printing the target validation data for encoder
val_target

array(['\tఅంక\n', '\tఅంక\n', '\tఅంకం\n', ..., '\tహోమేరిక్\n',
       '\tహోమేరిక్\n', '\tహోమేరిక్\n'], dtype='<U19')

In [ ]:
# Function to perform encoding of the characters 
def one_hot_encoding(input, target, val_input, val_target, input_tokens, target_tokens):

    input_token_index = dict([(char, i) for i, char in enumerate(input_tokens)])
    target_token_index = dict([(char, i) for i, char in enumerate(target_tokens)])

    reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
    reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

    
    encoder_input_data = np.zeros(
        (len(input), max_encoder_seq_length), dtype = "float32" )
    decoder_input_data = np.zeros(
        (len(input), max_decoder_seq_length), dtype = "float32" )
    decoder_target_data = np.zeros(
        (len(input), max_decoder_seq_length, num_decoder_tokens), dtype = "float32" )
    
    for i, (inp, tar) in enumerate(zip(input, target)):
        for t, char in enumerate(inp):
            encoder_input_data[i,t] = input_token_index[char]

        for t, char in enumerate(tar):
            decoder_input_data[i,t] = target_token_index[char]
            if t>0:
                decoder_target_data[i,t-1,target_token_index[char]] = 1.0

    

    encoder_val_input_data = np.zeros(
        (len(val_input), max_encoder_seq_length), dtype = "float32" )
    decoder_val_input_data = np.zeros(
        (len(val_input), max_decoder_seq_length), dtype = "float32" )
    decoder_val_target_data = np.zeros(
        (len(val_input), max_decoder_seq_length, num_decoder_tokens), dtype = "float32" )

    for i, (inp, tar) in enumerate(zip(val_input, val_target)):
        for t, char in enumerate(inp):
            encoder_val_input_data[i,t] = input_token_index[char]

        for t, char in enumerate(tar):
            decoder_val_input_data[i,t] = target_token_index[char]
            if t>0:
                decoder_val_target_data[i,t-1,target_token_index[char]] = 1.0

    return encoder_input_data, decoder_input_data, decoder_target_data, encoder_val_input_data, decoder_val_input_data, decoder_val_target_data


In [ ]:
encoder_input_data, decoder_input_data, decoder_target_data, encoder_val_input_data, decoder_val_input_data, decoder_val_target_data = one_hot_encoding(input, target, val_input, val_target, input_tokens, target_tokens)

# Cell functions

In [ ]:
def rnn( num_encoders, embed_size, dropout, num_decoders, hidden_layer_size):
  # enc_in : Encoder input
  # enc_out : Encoder output
  # enc_states: Encoder states
  # dec_in : Decoder input
  # dec_out : Decoder output
  # dec_dense : Dense layer for decoder

  #=============================================#
  #============== Encoder part =================#
  #=============================================#
  
  enc_in = Input(shape=(max_encoder_seq_length,))
  enc_out = Embedding(num_encoder_tokens, embed_size, trainable=True)(enc_in)

  enc_layers = []
  enc_states = []
  
  for i in range(num_encoders):
        encoder = SimpleRNN(hidden_layer_size, return_state=True, return_sequences=True, dropout=dropout)
        enc_layers.append(encoder)
        enc_out, state_h = encoder(enc_out)
        enc_states.append([state_h])

  #==============================================#  
  #============== Decoder part ==================#
  #==============================================#

  dec_in = Input(shape=(max_decoder_seq_length,))
  dec_out = Embedding(num_decoder_tokens, embed_size, trainable=True)(dec_in)

  dec_layers = []

  for i in range(num_decoders):
        decoder = SimpleRNN(hidden_layer_size, return_state=True, return_sequences=True, dropout=dropout)
        dec_layers.append(decoder)
        dec_out, _ = decoder(dec_out, initial_state = enc_states[i])
    
  dec_dense = TimeDistributed(Dense(num_decoder_tokens, activation="softmax"))
  dec_out = dec_dense(dec_out)
  
  model = Model([enc_in, dec_in], dec_out)

  return model, enc_layers, dec_layers


In [ ]:
def lstm( num_encoders, embed_size, dropout, num_decoders, hidden_layer_size):
  # e_in : Encoder input
  # e_out : Encoder output
  # e_states: Encoder states
  # d_in : Decoder input
  # d_out : Decoder output
  # d_dense : Dense layer for decoder


  #=============================================#
  #============== Encoder part =================#
  #=============================================#
  
  enc_in = Input(shape=(max_encoder_seq_length,))
  enc_out = Embedding(num_encoder_tokens, embed_size, trainable=True)(enc_in)

  enc_layers = []
  enc_states = []

  for i in range(num_encoders):
        encoder = LSTM(hidden_layer_size, return_state=True, return_sequences=True, dropout=dropout)
        enc_layers.append(encoder)
        enc_out, state_h, state_c = encoder(enc_out)
        enc_states.append([state_h, state_c])

  #==============================================#  
  #============== Decoder part ==================#
  #==============================================#


  dec_in = Input(shape=(max_decoder_seq_length,))
  dec_out = Embedding(num_decoder_tokens, embed_size, trainable=True)(dec_in)

  dec_layers = []

  for i in range(num_decoders):
        decoder = LSTM(hidden_layer_size, return_state=True, return_sequences=True, dropout=dropout)
        dec_layers.append(decoder)
        dec_out, _, _ = decoder(dec_out, initial_state = enc_states[i])

  dec_dense = TimeDistributed(Dense(num_decoder_tokens, activation="softmax"))
  dec_out = dec_dense(dec_out)
  
  model = Model([enc_in, dec_in], dec_out)

  return model, enc_layers, dec_layers


In [ ]:
def gru(num_encoders, embed_size, dropout, num_decoders, hidden_layer_size):
  # e_in : Encoder input
  # e_out : Encoder output
  # e_states: Encoder states
  # d_in : Decoder input
  # d_out : Decoder output
  # d_dense : Dense layer for decoder
  
 
  #=============================================#
  #============== Encoder part =================#
  #=============================================#
  

  enc_in = Input(shape=(max_encoder_seq_length,))
  enc_out = Embedding(num_encoder_tokens, embed_size, trainable=True)(enc_in)

  enc_layers = []
  enc_states = []

  for i in range(num_encoders):
        encoder = GRU(hidden_layer_size, return_state=True, return_sequences=True, dropout=dropout)
        enc_layers.append(encoder)
        enc_out, state_h = encoder(enc_out)
        enc_states.append([state_h])
  

  #==============================================#  
  #============== Decoder part ==================#
  #==============================================#


  dec_in = Input(shape=(max_decoder_seq_length,))
  dec_out = Embedding(num_decoder_tokens, embed_size, trainable=True)(dec_in)

  dec_layers = []

  for i in range(num_decoders):
        decoder = GRU(hidden_layer_size, return_state=True, return_sequences=True, dropout=dropout)
        dec_layers.append(decoder)
        dec_out, _ = decoder(dec_out, initial_state = enc_states[i])

    
  dec_dense =TimeDistributed(Dense(num_decoder_tokens, activation="softmax"))
  dec_out = dec_dense(dec_out)
  
  model = Model([enc_in, dec_in], dec_out)

  return model, enc_layers, dec_layers


In [ ]:
def build_model(num_encoders, num_decoders, cell, embed_size, dropout, hidden_layer_size):


  if cell == "RNN":
    model, enc_layers, dec_layers=rnn( num_encoders, embed_size, dropout, num_decoders, hidden_layer_size)
    return model, enc_layers, dec_layers
  elif cell == "LSTM":
    model, enc_layers, dec_layers=lstm(num_encoders, embed_size, dropout, num_decoders, hidden_layer_size)
    return model, enc_layers, dec_layers
  elif cell == "GRU":
    model, enc_layers, dec_layers=gru(num_encoders, embed_size, dropout, num_decoders, hidden_layer_size)
    return model, enc_layers, dec_layers
  


In [ ]:
from tqdm.notebook import tqdm_notebook

batch_size = 128        
epochs = 7             
input_embedding_size = 512
hidden_layer_size = 256
num_layers = 3
num_encoder_layers = num_layers
num_decoder_layers = num_layers
dropout = 0.2
cell_type = 'GRU'

config = {
    "num_encoders" : 3,
    "cell": "GRU",
    "embed_size": 512,
    "dropout": 0.2,
    "num_decoders" : 3,
    "hidden_layer_size":256
}

# TRAIN
model, encoder_layers, decoder_layers = build_model(num_encoder_layers, num_decoder_layers, cell_type, input_embedding_size, dropout, hidden_layer_size)

# COMPILE
model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])

# FIT
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    shuffle = True,
    validation_data= ([encoder_val_input_data, decoder_val_input_data], decoder_val_target_data)
)


Epoch 1/7
458/458 [==============================] - 27s 31ms/step - loss: 0.9745 - accuracy: 0.1771 - val_loss: 0.6203 - val_accuracy: 0.2377
Epoch 2/7
458/458 [==============================] - 12s 27ms/step - loss: 0.4466 - accuracy: 0.3152 - val_loss: 0.2901 - val_accuracy: 0.3316
Epoch 3/7
458/458 [==============================] - 13s 28ms/step - loss: 0.2198 - accuracy: 0.3826 - val_loss: 0.2175 - val_accuracy: 0.3538
Epoch 4/7
458/458 [==============================] - 13s 27ms/step - loss: 0.1417 - accuracy: 0.4057 - val_loss: 0.1468 - val_accuracy: 0.3761
Epoch 5/7
458/458 [==============================] - 13s 27ms/step - loss: 0.1054 - accuracy: 0.4166 - val_loss: 0.1426 - val_accuracy: 0.3790
Epoch 6/7
458/458 [==============================] - 13s 28ms/step - loss: 0.0838 - accuracy: 0.4233 - val_loss: 0.1415 - val_accuracy: 0.3801
Epoch 7/7
458/458 [==============================] - 13s 28ms/step - loss: 0.0693 - accuracy: 0.4276 - val_loss: 0.1383 - val_accuracy: 0.3819

NameError: ignored

# Inferencing model 

In [ ]:
def inferencing(model, num_encoders, num_decoders, enc_layers, dec_layers, cell, hidden_layer_size):

    enc_in = model.input[0]
    EmbeddingLayer = model.layers[2]    
    enc_out = EmbeddingLayer(enc_in)

    enc_states = []
    if cell == 'RNN':
        for i in range(num_encoders):
            enc_out, state_h = enc_layers[i](enc_out)
            enc_states += [state_h] 
    elif cell == 'LSTM':
        for i in range(num_encoders):
            enc_out, state_h, state_c = enc_layers[i](enc_out)
            enc_states += [state_h, state_c]   
    elif cell == 'GRU':
        for i in range(num_encoders):
            enc_out, state_h = enc_layers[i](enc_out)
            enc_states += [state_h] 

    encoder = Model(enc_in, enc_states + [enc_out])


    dec_in = model.input[1]    
    EmbeddingLayer = model.layers[3] 
    dec_out = EmbeddingLayer(dec_in)

    dec_states = []
    dec_initial_states = []
    
    if cell == 'RNN' :
        for i in range(num_decoders):
            dec_initial_states += [Input(shape=(hidden_layer_size,))]
            dec_out, state_h = dec_layers[i](dec_out, initial_state = dec_initial_states[i])
            dec_states += [state_h]
    elif cell == "LSTM":
        j=0
        for i in range(num_decoders):
            dec_initial_states += [Input(shape=(hidden_layer_size, )) , Input(shape=(hidden_layer_size, ))]
            dec_out, state_h, state_c = dec_layers[i](dec_out, initial_state=dec_initial_states[i+j:i+j+2])
            dec_states += [state_h , state_c]
            j += 1
    elif cell == "GRU":
        for i in range(num_decoders):
            dec_initial_states += [Input(shape=(hidden_layer_size,))]
            dec_out, state_h = dec_layers[i](dec_out, initial_state = dec_initial_states[i])
            dec_states += [state_h]

    dec_dense = model.layers[4 + 2*num_encoders]
    dec_out = dec_dense(dec_out)
    decoder = Model([dec_in] + dec_initial_states, [dec_out] + dec_states)

    return encoder, decoder 

In [ ]:
def decode_sequence(input_seq, encoder, decoder):

    states_value = encoder.predict(input_seq)
    states_value = states_value[:-1]

    target_seq = np.zeros((1, 1)) 
    target_seq[0, 0] = target_token_index["\t"]

    #print(input_seq)

    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens = decoder.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0][0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index
        
        states_value = output_tokens[1:]
        
    return decoded_sentence

In [ ]:
encoder_model, decoder_model = inferencing(model, num_encoder_layers, num_decoder_layers, encoder_layers, decoder_layers, cell_type, hidden_layer_size)
correct = 0
n = val_target.shape[0]
for i in range(20):
    inputs = encoder_input_data[i:i+1]
    output = decode_sequence(inputs,encoder_model, decoder_model)
    print("-")
    print("Input sentence:", input[i])
    print("Decoded sentence:", output)

-
Input sentence: amkita
Decoded sentence: అంకిత

-
Input sentence: ankita
Decoded sentence: అంకిత

-
Input sentence: ankitha
Decoded sentence: అంకిత

-
Input sentence: ankitam
Decoded sentence: అంకితరం

-
Input sentence: ankitham
Decoded sentence: అంకితరం

-
Input sentence: ankitabaavam
Decoded sentence: అంగికాషాంగా

-
Input sentence: ankithabhavam
Decoded sentence: అంకితరంచా

-
Input sentence: ankatamicchaadu
Decoded sentence: అంకితమిచచ్చి

-
Input sentence: ankitamicchadu
Decoded sentence: అంకితరించాడు

-
Input sentence: ankitamichhaadu
Decoded sentence: అంకితరించాడు

-
Input sentence: ankithamicchaadu
Decoded sentence: అంకితరించాడు

-
Input sentence: ankithamichhaadu
Decoded sentence: అంకితరించాడు

-
Input sentence: amkithamichaaru
Decoded sentence: అంకితమిచచ్చి

-
Input sentence: ankithamichaaru
Decoded sentence: అంకితమిచచ్చి

-
Input sentence: ankithamicharu
Decoded sentence: అంకితరించారు

-
Input sentence: ankithamaina
Decoded sentence: అంకితమించాయి

-
Input sentence: ankusham
D

In [ ]:
def train_model(config, encoder_input, decoder_input, decoder_target, encoder_val_input, decoder_val_input, decoder_val_target ):

    batch_size = config['batch_size']
    cell = config['cell']
    embed_size = config['embed_size']
    dropout = config['dropout']
    num_encoders = config['num_encoders']
    num_decoders = config['num_decoders']
    hidden_layer_size = config['hidden_layer_size']
    epochs = config['epochs']
    optimizer = config['optimizer']


    model, encoder_layers, decoder_layers = build_model(num_encoders, num_decoders, cell, embed_size, dropout, hidden_layer_size)

    model.compile(optimizer = optimizer, loss = "categorical_crossentropy", metrics=["accuracy"])

    model.fit(
    [encoder_input, decoder_input],
    decoder_target,
    batch_size = batch_size,
    epochs = epochs,
    shuffle = True,
    validation_data= ([encoder_val_input, decoder_val_input], decoder_val_target)
    )

    return model


# Wandb Sweeps 

In [ ]:
# Enter the project and the entity name for wandb sweeps
project_name = "Assignment 3"
entity_name = "safikhan"


In [ ]:
def train():
  default_config={
      "cell": "RNN",
      "embed_size":256,
      "dropout":0.2,
      "num_layers": 1,
      "hidden_layer_size":128,
      "epochs": 1,
      "batch_size": 64,
      "optimizer": 'adam'
  }

  wandb.init(config=default_config, project=project_name, entity=entity_name)
  config = wandb.config
  wandb.run.name = str(config.cell)\
      +"_"+str(config.embed_size)\
      +"_"+str(config.dropout)\
      +"_"+str(config.num_layers)\
      +"_"+str(config.num_layers)\
      +"_"+str(config.hidden_layer_size)\
      +"_"+str(config.epochs)\
      +"_"+str(config.batch_size)

  #wandb.run.save()
  
  
  batch_size = config['batch_size']
  cell = config['cell']
  embed_size = config['embed_size']
  dropout = config['dropout']
  num_encoders = config['num_layers']
  num_decoders = config['num_layers']
  hidden_layer_size = config['hidden_layer_size']
  epochs = config['epochs']
  optimizer = config['optimizer']


  model, encoder_layers, decoder_layers = build_model(num_encoders, num_decoders, cell, embed_size, dropout, hidden_layer_size)

  
  model.compile(
      optimizer = optimizer,
      loss = tf.keras.losses.CategoricalCrossentropy(name='loss'),
      metrics = [tf.keras.metrics.CategoricalAccuracy(name='acc')]
  )
  
  model.fit(
      [encoder_input_data, decoder_input_data],
      decoder_target_data,
      batch_size = batch_size,
      epochs = epochs,
      shuffle = True,
      callbacks=[WandbCallback()],
      validation_data= ([encoder_val_input_data, decoder_val_input_data], decoder_val_target_data)
  )

  wandb.run.finish()



  

In [ ]:
hyperparameters = {

          "cell":{
              "values":["RNN","GRU","LSTM"]
              },

          "embed_size":{
              "values":[16,32,64,256]
              },

          "hidden_layer_size":{
              "values":[16,32,64,256]
              },

          "num_layers":{
              "values":[1,2,3]
              },

          "dropout":{
              "values":[0.2,0.3,0.4]
              },

          "epochs":{
              "values":[5,10,15,20]
              },

          "batch_size":{
              "values":[32,64]
              },

          "optimizer":{
              "values":["adam", "rmsprop", "nadam"]
              }
      }

In [ ]:
def sweeper(project_name,entity_name):
  sweep_config={

      "method": "bayes",
      "metric": {
          "name": "val_acc", 
          "goal": "maximize"
          },
      "parameters":hyperparameters
  }

  sweep_id=wandb.sweep(sweep_config, project=project_name, entity=entity_name)
  wandb.agent(sweep_id,train)


In [ ]:
# Running the sweeps for hyperparameter tuning
sweeper(project_name,entity_name)

Create sweep with ID: 5yix48o9
Sweep URL: https://wandb.ai/safikhan/Assignment%203/sweeps/5yix48o9


wandb: Agent Starting Run: d879x8a5 with config:
wandb: 	batch_size: 64
wandb: 	cell: GRU
wandb: 	dropout: 0.3
wandb: 	embed_size: 64
wandb: 	epochs: 10
wandb: 	hidden_layer_size: 16
wandb: 	num_layers: 1
wandb: 	optimizer: rmsprop
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Epoch 1/10
915/915 [==============================] - 10s 8ms/step - loss: 1.3180 - acc: 0.0967 - val_loss: 1.1105 - val_acc: 0.1195 - _timestamp: 1651165392.0000 - _runtime: 16.0000
Epoch 2/10
915/915 [==============================] - 8s 9ms/step - loss: 1.1371 - acc: 0.1330 - val_loss: 1.0278 - val_acc: 0.1436 - _timestamp: 1651165400.0000 - _runtime: 24.0000
Epoch 3/10
915/915 [==============================] - 10s 11ms/step - loss: 1.0804 - acc: 0.1498 - val_loss: 0.9829 - val_acc: 0.1517 - _timestamp: 1651165410.0000 - _runtime: 34.0000
Epoch 4/10
915/915 [==============================] - 7s 8ms/step - loss: 1.0444 - acc: 0.1579 - val_loss: 0.9559 - val_acc: 0.1572 - _timestamp: 1651165417.0000 - _runtime: 41.0000
Epoch 5/10
915/915 [==============================] - 10s 11ms/step - loss: 1.0226 - acc: 0.1622 - val_loss: 0.9395 - val_acc: 0.1587 - _timestamp: 1651165427.0000 - _runtime: 51.0000
Epoch 6/10
915/915 [==============================] - 9s 10ms/step - loss: 1.0060 - a

acc,▁▄▅▆▆▇▇▇██
epoch,▁▂▃▃▄▅▆▆▇█
loss,█▅▃▃▂▂▂▁▁▁
val_acc,▁▄▅▅▆▆▇▇▇█
val_loss,█▅▄▃▃▂▂▂▁▁
acc,0.18223
best_epoch,9
best_val_loss,0.87899
epoch,9
loss,0.95322
val_acc,0.17832


wandb: Agent Starting Run: q4bkjd8p with config:
wandb: 	batch_size: 64
wandb: 	cell: RNN
wandb: 	dropout: 0.3
wandb: 	embed_size: 256
wandb: 	epochs: 10
wandb: 	hidden_layer_size: 256
wandb: 	num_layers: 3
wandb: 	optimizer: adam
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


...

acc,▁▃▅▆▆▇▇▇▇███████████
epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
loss,█▅▄▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
val_acc,▁▄▅▆▆▇▇▇████████████
val_loss,█▅▄▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
acc,0.43452
best_epoch,13
best_val_loss,0.21402
epoch,19
loss,0.04525
val_acc,0.368


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: vati32mu with config:
wandb: 	batch_size: 32
wandb: 	cell: LSTM
wandb: 	dropout: 0.2
wandb: 	embed_size: 256
wandb: 	epochs: 20
wandb: 	hidden_layer_size: 256
wandb: 	num_layers: 1
wandb: 	optimizer: rmsprop
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Epoch 1/20
1830/1830 [==============================] - 24s 12ms/step - loss: 0.8269 - acc: 0.2137 - val_loss: 0.4110 - val_acc: 0.2948 - _timestamp: 1651218690.0000 - _runtime: 37.0000
Epoch 2/20
1830/1830 [==============================] - 20s 11ms/step - loss: 0.2730 - acc: 0.3666 - val_loss: 0.2056 - val_acc: 0.3577 - _timestamp: 1651218710.0000 - _runtime: 57.0000
Epoch 3/20
1830/1830 [==============================] - 20s 11ms/step - loss: 0.1516 - acc: 0.4030 - val_loss: 0.1630 - val_acc: 0.3715 - _timestamp: 1651218730.0000 - _runtime: 77.0000
Epoch 4/20
1830/1830 [==============================] - 20s 11ms/step - loss: 0.1087 - acc: 0.4161 - val_loss: 0.1536 - val_acc: 0.3743 - _timestamp: 1651218750.0000 - _runtime: 97.0000
Epoch 5/20
1830/1830 [==============================] - 20s 11ms/step - loss: 0.0847 - acc: 0.4235 - val_loss: 0.1524 - val_acc: 0.3760 - _timestamp: 1651218769.0000 - _runtime: 116.0000
Epoch 6/20
1830/1830 [==============================] - 20s 11ms/step

acc,▁▆▇▇▇███████████████
epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
loss,█▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_acc,▁▆▇█████████████████
val_loss,█▂▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂▂
acc,0.44257
best_epoch,5
best_val_loss,0.15034
epoch,19
loss,0.02091
val_acc,0.37692


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: wnmo0bqg with config:
wandb: 	batch_size: 32
wandb: 	cell: LSTM
wandb: 	dropout: 0.2
wandb: 	embed_size: 256
wandb: 	epochs: 20
wandb: 	hidden_layer_size: 64
wandb: 	num_layers: 3
wandb: 	optimizer: adam
wandb: W&B API key is configured (use `wandb login --relogin` to force relogin)
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


# Testing the model on test dataset

In [ ]:
def load_test_data():
    path = r"/content/drive/MyDrive/dakshina_dataset_v1.0/dakshina_dataset_v1.0/te/lexicons/"

    test_file_path = path + "te.translit.sampled.test.tsv"

    

    test_tsv_file = open(test_file_path, "r", encoding="utf-8")
    test_dataset = csv.reader(test_tsv_file, delimiter="\t")

    return test_dataset


In [ ]:
def prepare_test_data():

    test_dataset = load_test_data()
    
    test_input = []
    test_target = []
    #print(english)
    for i in test_dataset:
        #print(i) 
        test_target.append(i[0])
        #print(i[1])
        test_input.append(i[1])
    #print(english)
    target = np.array(test_target)
    input = np.array(test_input)

    for i in range(len(target)):
        test_target[i] = "\t" + target[i] + "\n"
    


    return test_input, test_target


In [ ]:
test_input, test_target = prepare_test_data()

In [ ]:
def one_hot_encoding_test(input, target, input_tokens, target_tokens):

    input_token_index = dict([(char, i) for i, char in enumerate(input_tokens)])
    target_token_index = dict([(char, i) for i, char in enumerate(target_tokens)])

    reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
    reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

    
    encoder_input_data = np.zeros(
        (len(input), max_encoder_seq_length), dtype = "float32" )
    decoder_input_data = np.zeros(
        (len(input), max_decoder_seq_length), dtype = "float32" )
    decoder_target_data = np.zeros(
        (len(input), max_decoder_seq_length, num_decoder_tokens), dtype = "float32" )
    
    for i, (inp, tar) in enumerate(zip(input, target)):
        for t, char in enumerate(inp):
            encoder_input_data[i,t] = input_token_index[char]

        for t, char in enumerate(tar):
            decoder_input_data[i,t] = target_token_index[char]
            if t>0:
                decoder_target_data[i,t-1,target_token_index[char]] = 1.0


    return encoder_input_data, decoder_input_data, decoder_target_data

In [ ]:
encoder_test_input_data, decoder_test_input_data, decoder_test_target_data = one_hot_encoding_test(test_input, test_target, input_tokens, target_tokens)

In [ ]:
best_hyperparameters = {
      "cell": "LSTM",
      "embed_size":256,
      "dropout":0.3,
      "num_layers": 3,
      "hidden_layer_size":256,
      "epochs": 20,
      "batch_size": 32,
      "optimizer": 'adam'
}

In [ ]:
def testing():
  wandb.init(config=best_hyperparameters, project=project_name, entity=entity_name)
  config = wandb.config
  wandb.run.name = "TestRun - without attention"

  #wandb.run.save()
  
  
  batch_size = config['batch_size']
  cell = config['cell']
  embed_size = config['embed_size']
  dropout = config['dropout']
  num_encoders = config['num_layers']
  num_decoders = config['num_layers']
  hidden_layer_size = config['hidden_layer_size']
  epochs = config['epochs']
  optimizer = config['optimizer']


  model, encoder_layers, decoder_layers = build_model(num_encoders, num_decoders, cell, embed_size, dropout, hidden_layer_size)

  
  model.compile(
      optimizer = optimizer,
      loss = tf.keras.losses.CategoricalCrossentropy(name='loss'),
      metrics = [tf.keras.metrics.CategoricalAccuracy(name='acc')]
  )
  
  model.fit(
      [encoder_input_data, decoder_input_data],
      decoder_target_data,
      batch_size = batch_size,
      epochs = epochs,
      shuffle = True,
      callbacks=[WandbCallback()],
      validation_data= ([encoder_test_input_data, decoder_test_input_data], decoder_test_target_data)
  )

  return model, encoder_layers, decoder_layers

In [ ]:
model, encoder_layers, decoder_layers = testing()
encoder_model, decoder_model = inferencing(model, best_hyperparameters['num_layers'], best_hyperparameters['num_layers'], encoder_layers, decoder_layers, best_hyperparameters['cell'], best_hyperparameters['hidden_layer_size'])
correct = 0
n = test_input.shape[0]
rows = []
for i in range(20):
    inputs = encoder_test_input_data[i:i+1]
    output = decode_sequence(inputs,encoder_model, decoder_model)
    point = [input, output]
    rows.append(point)
    print("-")
    print("Input sentence:", test_input[i])
    print("Decoded sentence:", output)



<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch 1/20
1830/1830 [==============================] - 49s 20ms/step - loss: 1.0088 - acc: 0.1643 - val_loss: 0.7144 - val_acc: 0.2198 - _timestamp: 1651231694.0000 - _runtime: 60.0000
Epoch 2/20
1830/1830 [==============================] - 34s 19ms/step - loss: 0.5811 - acc: 0.2769 - val_loss: 0.4195 - val_acc: 0.2984 - _timestamp: 1651231728.0000 - _runtime: 94.0000
Epoch 3/20
1830/1830 [==============================] - 35s 19ms/step - loss: 0.3088 - acc: 0.3551 - val_loss: 0.2185 - val_acc: 0.3586 - _timestamp: 1651231764.0000 - _runtime: 130.0000
Epoch 4/20
1830/1830 [==============================] - 35s 19ms/step - loss: 0.1847 - acc: 0.3918 - val_loss: 0.1685 - val_acc: 0.3732 - _timestamp: 1651231799.0000 - _runtime: 165.0000
Epoch 5/20
1830/1830 [==============================] - 35s 19ms/step - loss: 0.1356 - acc: 0.4068 - val_loss: 0.1483 - val_acc: 0.3791 - _timestamp: 1651231833.0000 - _runtime: 199.0000
Epoch 6/20
1830/1830 [==============================] - 35s 19ms/st

AttributeError: ignored

In [ ]:
#n = test_input.shape[0]
rows = []
for i in range(20):
    inputs = encoder_test_input_data[i:i+1]
    output = decode_sequence(inputs,encoder_model, decoder_model)
    point = [input, output]
    rows.append(point)
    print("-")
    print("Input sentence:", test_input[i])
    print("Decoded sentence:", output)

-
Input sentence: amkamlo
Decoded sentence: అంకంలో

-
Input sentence: ankamlo
Decoded sentence: అంకంలో

-
Input sentence: ankamloo
Decoded sentence: అంకంలో

-
Input sentence: amkitamai
Decoded sentence: అంకితమై

-
Input sentence: ankitamai
Decoded sentence: అంకితమై

-
Input sentence: ankela
Decoded sentence: అంకెల

-
Input sentence: ankelanu
Decoded sentence: అంకెలను

-
Input sentence: angeekarinchaka
Decoded sentence: అంగీకరించాక

-
Input sentence: amgiikarimchaadu
Decoded sentence: అంగీకరించాడు

-
Input sentence: angeekarinchaadu
Decoded sentence: అంగీకరించాడు

-
Input sentence: angiikarinchaadu
Decoded sentence: అంగీకరించాడు

-
Input sentence: angeekarinchaali
Decoded sentence: అంగీకరించాలి

-
Input sentence: angikarinchaali
Decoded sentence: అంగీకరించాలి

-
Input sentence: angeekarinchi
Decoded sentence: అంగీకరించి

-
Input sentence: angikarinchi
Decoded sentence: అంగీకరించి

-
Input sentence: angeekarimchina
Decoded sentence: అంగీకరించిన

-
Input sentence: angeekarinchina
Decoded 

In [ ]:
best_hyperparameters = {
      "cell": "GRU",
      "embed_size":256,
      "dropout":0.4,
      "num_layers": 3,
      "hidden_layer_size":256,
      "epochs": 10,
      "batch_size": 32,
      "optimizer": 'adam'
}

In [ ]:
model1, encoder_layers1, decoder_layers1 = testing()
encoder_model1, decoder_model1 = inferencing(model1, best_hyperparameters['num_layers'], best_hyperparameters['num_layers'], encoder_layers1, decoder_layers1, best_hyperparameters['cell'], best_hyperparameters['hidden_layer_size'])
correct = 0
#n = test_input.shape[0]
rows = []
for i in range(20):
    inputs = encoder_test_input_data[i:i+1]
    output = decode_sequence(inputs,encoder_model1, decoder_model1)
    point = [input, output]
    rows.append(point)
    print("-")
    print("Input sentence:", test_input[i])
    print("Decoded sentence:", output)


Epoch 1/10
1830/1830 [==============================] - 43s 19ms/step - loss: 0.6301 - acc: 0.2679 - val_loss: 0.2289 - val_acc: 0.3555 - _timestamp: 1651233387.0000 - _runtime: 52.0000
Epoch 2/10
1830/1830 [==============================] - 35s 19ms/step - loss: 0.2038 - acc: 0.3862 - val_loss: 0.1657 - val_acc: 0.3743 - _timestamp: 1651233421.0000 - _runtime: 86.0000
Epoch 3/10
1830/1830 [==============================] - 34s 19ms/step - loss: 0.1469 - acc: 0.4030 - val_loss: 0.1536 - val_acc: 0.3788 - _timestamp: 1651233455.0000 - _runtime: 120.0000
Epoch 4/10
1830/1830 [==============================] - 34s 18ms/step - loss: 0.1223 - acc: 0.4105 - val_loss: 0.1482 - val_acc: 0.3815 - _timestamp: 1651233489.0000 - _runtime: 154.0000
Epoch 5/10
1830/1830 [==============================] - 34s 18ms/step - loss: 0.1084 - acc: 0.4147 - val_loss: 0.1470 - val_acc: 0.3828 - _timestamp: 1651233523.0000 - _runtime: 188.0000
Epoch 6/10
1830/1830 [==============================] - 34s 18ms/st